In [1]:
import sys
import os
sys.path.append(os.path.abspath('../..'))



In [6]:

# from scan_engine.status_fields_config_24summ import STATUS_FIELDS_CONFIG
from status_fields_config_oct3v1_brws_250220 import STATUS_FIELDS_CONFIG

from utlis.scan_engine_utlis.scan_eng_big_utlis import log_folder_to_parquet_sep

if __name__ == "__main__":
    base_folder = "/data/big_rim/rsync_dcc_sum/Oct3V1" #"/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ"  # Replace with your base folder
    # save_path = os.path.join(base_folder, 'paret')
    failed_paths_file = '/data/big_rim/rsync_dcc_sum/Oct3V1/sync_failed_brws.txt'  # File containing failed paths


    force_rescan_rec_files = [
        # ('2023-10-01', '001'),
        # ('2023-10-02', '002'),
        # Add more as needed
    ]
    rescan_threshold_days = 0.001 # 7 days, but guess if i mess up i can just change it to automatically rescan all, smile... #0.1

    log_folder_to_parquet_sep(base_folder, failed_paths_file, STATUS_FIELDS_CONFIG,
                              force_rescan_rec_files=force_rescan_rec_files,
                              rescan_threshold_days=rescan_threshold_days)


Log for 20240819V1r2 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03/20240819V1r2/folder_log.parquet
Log for 24Anshu_f_paint_2mice_2 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_paint_2mice_2/folder_log.parquet
Log for 24Anshu_f_paint_2mice saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_paint_2mice/folder_log.parquet
Log for 24Anshu_f_bleach_2mice saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_03_micecolor_test/24Anshu_f_bleach_2mice/folder_log.parquet
Log for 20241001PMCRE2mini_15_22 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_02_12/20241001PMCRE2mini_15_22/folder_log.parquet
Log for 20241001PMCRE2mini_13_57 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_02_12/20241001PMCRE2mini_13_57/folder_log.parquet
Log for 20241001PMCRE2mini_13_44 saved at /data/big_rim/rsync_dcc_sum/Oct3V1/2025_02_12/20241001PMCRE2mini_13_44/folder_log.parquet
Log for 20241001PMCRE2mini_15_35 saved at /data/big_rim/rsync_dcc_s

In [3]:
# sys.path.append(os.path.abspath('../..'))
from utlis.scan_engine_utlis.scan_engine_utlis import read_all_parquet_files
# base_folder = "/hpc/group/tdunn/Bryan_Rigs/BigOpenField/24summ"
all_df = read_all_parquet_files(base_folder)

In [4]:
import os
import datetime
import concurrent.futures
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

from utlis.sync_utlis.sync_df_utlis import find_calib_file
from utlis.scan_engine_utlis.scan_engine_utlis import (
    read_failed_paths,
    match_date_pattern,
    assign_status_codes,
)
from status_fields_config_oct3v1_brws_250220 import STATUS_FIELDS_CONFIG

from utlis.scan_engine_utlis.scan_log_utlis import (
    load_scan_log,
    save_scan_log,
    clean_scan_log,
    update_scan_log,
)


def process_experiment_dir(animal_id, custom_label, date_str, time_str, root_dir, failed_entries, config):
    """
    Process a single experiment directory following:
    root_dir / animal_id / custom_label / date_str / time_str

    This function:
      - Identifies calibration files (those starting with 'calib') at the experiment level.
      - Iterates over experiment subdirectories (recording folders).
      - For each rec folder, it calls `find_calib_file` and `assign_status_codes`,
        and adds extra fields such as animal_id, custom_label, date, and time.
    """
    exp_path = os.path.join(root_dir, animal_id, custom_label, date_str, time_str)
    rec_entries = []
    calib_files = []

    # Identify calibration files within the experiment folder
    for file_name in os.listdir(exp_path):
        if file_name.startswith("calib"):
            calib_files.append(file_name)

    # Scan for rec (recording) folders
    for rec_folder in os.listdir(exp_path):
        rec_path = os.path.join(exp_path, rec_folder)
        if os.path.isdir(rec_path):
            # Detect associated calibration file (if any)
            calib_file = find_calib_file(rec_path)
            rec_entry = assign_status_codes(
                animal_id, rec_path, calib_file, failed_entries, config
            )
            rec_entry['rec_name'] = rec_folder
            rec_entry['timestamp'] = datetime.datetime.now().isoformat()
            rec_entry['animal_id'] = animal_id
            rec_entry['custom_label'] = custom_label
            rec_entry['date'] = date_str
            rec_entry['time'] = time_str
            rec_entries.append(rec_entry)

    return {
        'animal_id': animal_id,
        'custom_label': custom_label,
        'date': date_str,
        'time': time_str,
        'calib_files': calib_files,
        'rec_entries': rec_entries
    }


def log_experiment_data(root_dir, failed_paths_file, config, force_reprocess=None, rescan_days=7):
    """
    Scan experiments and log data, saving Parquet files per rec folder.

    Assumes folder structure:
      root_dir/
          animal_id (e.g., "20240910-V1-R")/
              custom_label (e.g., "customEntValHere")/
                  date (e.g., "2024_11_13")/
                      time (e.g., "16_18_24")/
                          rec_folder (e.g., "My_V4_Miniscope")

    Generates a `folder_log.parquet` inside each rec folder.
    The scan log is updated using "animal_id/custom_label/date/time".
    """
    scan_log_path = os.path.join(root_dir, 'paret', 'scan_log.csv')
    scan_log_df = load_scan_log(scan_log_path)

    # Read manually recorded failed paths
    failed_entries = read_failed_paths(failed_paths_file) if failed_paths_file else set()

    if force_reprocess is None:
        force_reprocess = []
    force_reprocess_set = set(force_reprocess)

    # Identify experiments needing a scan
    experiments_to_process = []
    for animal_id in os.listdir(root_dir):
        animal_path = os.path.join(root_dir, animal_id)
        if not os.path.isdir(animal_path):
            continue
        for custom_label in os.listdir(animal_path):
            custom_path = os.path.join(animal_path, custom_label)
            if not os.path.isdir(custom_path):
                continue
            for date_str in os.listdir(custom_path):
                date_path = os.path.join(custom_path, date_str)
                if not os.path.isdir(date_path) or not match_date_pattern(date_str):
                    continue
                for time_str in os.listdir(date_path):
                    time_path = os.path.join(date_path, time_str)
                    if not os.path.isdir(time_path):
                        continue
                    exp_key = (animal_id, custom_label, date_str, time_str)
                    experiments_to_process.append(exp_key)

    if not experiments_to_process:
        print("No new or modified experiments to process.")
        return

    # Parallel processing of experiment directories
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for animal_id, custom_label, date_str, time_str in experiments_to_process:
            futures.append(
                executor.submit(
                    process_experiment_dir,
                    animal_id, custom_label, date_str, time_str, root_dir, failed_entries, config
                )
            )

        for future in concurrent.futures.as_completed(futures):
            exp_data = future.result()
            animal_id = exp_data['animal_id']
            custom_label = exp_data['custom_label']
            date_str = exp_data['date']
            time_str = exp_data['time']
            calib_files = exp_data.get('calib_files', [])
            calib_files = [str(f) for f in calib_files] if calib_files else []

            # Process and save logs per recording
            for rec_entry in exp_data['rec_entries']:
                rec_name = rec_entry['rec_name']
                save_path = os.path.join(root_dir, animal_id, custom_label, date_str, time_str, rec_name, "folder_log.parquet")
                os.makedirs(os.path.dirname(save_path), exist_ok=True)

                # Attach calibration info
                rec_entry['calib_files'] = calib_files

                # Convert status fields to string based on config
                status_columns = list(config.keys())
                df = pd.DataFrame([rec_entry])
                df[status_columns] = df[status_columns].astype(str)

                table = pa.Table.from_pandas(df)
                pq.write_table(table, save_path)
                print(f"Log for {rec_name} saved at {save_path}")

                # Update scan log with a key like "animal/custom/date/time"
                scan_key = f"{animal_id}/{custom_label}/{date_str}/{time_str}"
                scan_log_df = update_scan_log(scan_log_df, scan_key, rec_name)

    # Finalize scan log
    scan_log_df = clean_scan_log(scan_log_df, root_dir)
    save_scan_log(scan_log_df, scan_log_path)


if __name__ == "__main__":
    root_dir = "/data/big_rim/mir_data/Oct3V1mini/Oct3V1mini_sorted"
    failed_paths_file = '/data/big_rim/mir_data/Oct3V1mini/Oct3V1mini_sorted/sync_failed.txt'

    force_reprocess = []

    rescan_days = 0.001  # Lower for testing

    log_experiment_data(root_dir, failed_paths_file, STATUS_FIELDS_CONFIG,
                        force_reprocess=force_reprocess,
                        rescan_days=rescan_days)


Log for My_V4_Miniscope saved at /data/big_rim/mir_data/Oct3V1mini/Oct3V1mini_sorted/20240819-V1-r1/customEntValHere/2024_10_04/21_41_08/My_V4_Miniscope/folder_log.parquet
Log for My_First_WebCam saved at /data/big_rim/mir_data/Oct3V1mini/Oct3V1mini_sorted/20240819-V1-r1/customEntValHere/2024_10_04/21_41_08/My_First_WebCam/folder_log.parquet
Log for My_V4_Miniscope saved at /data/big_rim/mir_data/Oct3V1mini/Oct3V1mini_sorted/20240919V1L5/customEntValHere/2024_10_31/13_14_16/My_V4_Miniscope/folder_log.parquet
Log for My_First_WebCam saved at /data/big_rim/mir_data/Oct3V1mini/Oct3V1mini_sorted/20240919V1L5/customEntValHere/2024_10_31/13_14_16/My_First_WebCam/folder_log.parquet
Log for My_V4_Miniscope saved at /data/big_rim/mir_data/Oct3V1mini/Oct3V1mini_sorted/20240910-V1-R/customEntValHere/2024_11_13/16_18_24/My_V4_Miniscope/folder_log.parquet
Log for My_First_WebCam saved at /data/big_rim/mir_data/Oct3V1mini/Oct3V1mini_sorted/20240910-V1-R/customEntValHere/2024_11_13/16_18_24/My_First_

OSError: Cannot save file into a non-existent directory: '/data/big_rim/mir_data/Oct3V1mini/Oct3V1mini_sorted/paret'

In [ ]:
import pyarrow.compute as pc
table = all_df #combined_df
# Filter mir_generate_param == 0 and sync != 3
filter_mask = pc.and_(
    pc.equal(table['mir_generate_param'], '0'), 
    pc.not_equal(table['sync'], '3')
)

# Apply the filter and print the results
filtered_table = table.filter(filter_mask)

# Print each row of the filtered table
print(filtered_table.to_pandas())  # This will display the filtered data in a familiar pandas-like format
